
# ROI-based decoding analysis in Haxby et al. dataset

In this script we reproduce the data analysis
conducted by :footcite:t:`Haxby2001`.

Specifically, we look at decoding accuracy for different objects in
three different masks: the full ventral stream (mask_vt), the house
selective areas (mask_house) and the face selective areas (mask_face),
that have been defined via a standard GLM-based analysis.

.. include:: ../../../examples/masker_note.rst


In [1]:
# Fetch data using nilearn dataset fetcher
import os
from nilearn import datasets
from nilearn.plotting import show

## Load and prepare the data



In [19]:
# by default we fetch 2nd subject data for analysis
#haxby_dataset = datasets.fetch_haxby()
root_dir = '/Volumes/T7/BIDS-BRAINPLAYBACK-TASK2'
fmriprep_dir = os.path.join(root_dir, 'derivatives', 'fmriprep23')
func_filename = os.path.join(fmriprep_dir, 'sub-01', 'ses-01', 'func', 'sub-01_ses-01_task-02a_run-1_space-MNI152NLin2009cAsym_res-2_desc-preproc_bold.nii.gz')

# load labels
import pandas as pd
import numpy as np

# Load nilearn NiftiMasker, the practical masking and unmasking tool
from nilearn.maskers import NiftiMasker

# import events data
from functions import edit_events
events = edit_events(root_dir, 'sub-01', '02a', '1')
events.reset_index(drop=True, inplace=True)
#labels = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
#stimuli = labels["labels"]
# identify resting state labels in order to be able to remove them
#task_mask = events['trial_type'] != "Noise"

# create a list the length of the number of volumes in the functional image
# with the trial type (string) of each volume
task_per_volume = 
for i in range(len(events)):
    task_per_volume[events['onset']:events['onset'][i]+events['duration'][i]] = events['trial_type'][i]
# find names of remaining active labels
#categories = events['trial_type'][task_mask].unique()

# extract tags indicating to which acquisition run a tag belongs
#run_labels = labels["chunks"][task_mask]

# apply the task_mask to  fMRI data (func_filename)
#from nilearn.image import index_img

#task_data = index_img(func_filename, task_mask)

TypeError: slice indices must be integers or None or have an __index__ method

In [20]:
events['trial_type'][1]

'Music'

12

In [14]:
events

,onset,duration,trial_type
1,12,24,Music
3,42,24,Music
4,66,18,Noise
5,84,24,Music
7,114,24,Music
8,138,18,Noise
9,156,24,Music
11,187,24,Music
12,210,18,Noise
13,228,24,Music


In [ ]:
task_per_volume

## Decoding on the different masks

The classifier used here is a support vector classifier (svc). We use
class:`nilearn.decoding.Decoder` and specify the classifier.



In [ ]:
import numpy as np

# Make a data splitting object for cross validation
from sklearn.model_selection import LeaveOneGroupOut

from nilearn.decoding import Decoder

cv = LeaveOneGroupOut()

We use :class:`nilearn.decoding.Decoder` to estimate a baseline.



In [ ]:
mask_names = ["mask_vt", "mask_face", "mask_house"]

mask_scores = {}
mask_chance_scores = {}

for mask_name in mask_names:
    print(f"Working on {mask_name}")
    # For decoding, standardizing is often very important
    mask_filename = haxby_dataset[mask_name][0]
    masker = NiftiMasker(mask_img=mask_filename, standardize="zscore_sample")
    mask_scores[mask_name] = {}
    mask_chance_scores[mask_name] = {}

    for category in categories:
        print(f"Processing {mask_name} {category}")
        classification_target = stimuli[task_mask] == category
        # Specify the classifier to the decoder object.
        # With the decoder we can input the masker directly.
        # We are using the svc_l1 here because it is intra subject.
        decoder = Decoder(
            estimator="svc_l1",
            cv=cv,
            mask=masker,
            scoring="roc_auc",
            standardize="zscore_sample",
        )
        decoder.fit(task_data, classification_target, groups=run_labels)
        mask_scores[mask_name][category] = decoder.cv_scores_[1]
        mean = np.mean(mask_scores[mask_name][category])
        std = np.std(mask_scores[mask_name][category])
        print(f"Scores: {mean:1.2f} +- {std:1.2f}")

        dummy_classifier = Decoder(
            estimator="dummy_classifier",
            cv=cv,
            mask=masker,
            scoring="roc_auc",
            standardize="zscore_sample",
        )
        dummy_classifier.fit(
            task_data, classification_target, groups=run_labels
        )
        mask_chance_scores[mask_name][category] = dummy_classifier.cv_scores_[
            1
        ]

## We make a simple bar plot to summarize the results



In [ ]:
import matplotlib.pyplot as plt

plt.figure()

tick_position = np.arange(len(categories))
plt.xticks(tick_position, categories, rotation=45)

for color, mask_name in zip("rgb", mask_names):
    score_means = [
        np.mean(mask_scores[mask_name][category]) for category in categories
    ]
    plt.bar(
        tick_position, score_means, label=mask_name, width=0.25, color=color
    )

    score_chance = [
        np.mean(mask_chance_scores[mask_name][category])
        for category in categories
    ]
    plt.bar(
        tick_position,
        score_chance,
        width=0.25,
        edgecolor="k",
        facecolor="none",
    )

    tick_position = tick_position + 0.2

plt.ylabel("Classification accuracy (AUC score)")
plt.xlabel("Visual stimuli category")
plt.ylim(0.3, 1)
plt.legend(loc="lower right")
plt.title("Category-specific classification accuracy for different masks")
plt.tight_layout()

show()

## References

 .. footbibliography::

